In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from numba import njit, jit

In [ ]:
d=2

N=4
sigma = 0
k = 10
noise_scale =0.001


t_max = 100
time_steps = 10 +1

t, dt = np.linspace(0,t_max, time_steps,retstep=True)




def f(t, y):
    r = y[:d*N].reshape([-1,d])
    print(t)
    print(r)
    v = y[d*N:].reshape([-1,d])
    eta = np.random.normal(0,noise_scale,size=[N*d]).reshape([N,d])
    dv = -sigma*v + k*(np.roll(r, 1, axis=0) + np.roll(r, -1, axis=0) - 2*r) + eta
    
    dv[0] = -sigma*v[0] + k*(r[1] - r[0]) + eta[0]
    dv[-1] = -sigma*v[-1] + k*(r[-2] - r[-1]) + eta[-1]

    dr = v
    dy = np.stack((dr,dv)).flatten()
    print(dy)
    return dy



y0 = np.zeros([2*N*d])

x0 = np.linspace(0,1,N)
for i in range(0,N):
    y0[i*d] = x0[i]
print(y0)
sol = solve_ivp(f, (0,t_max), y0, t_eval=t)

y = sol.y.T

print(y.shape)
r = y[:, :d*N].reshape([time_steps, -1,d])
v = y[:, d*N:].reshape([time_steps, -1,d])
for t in range(time_steps):
    plt.scatter(r[t,:,0], r[t,:,1])
    plt.show()
print(sol)

